In [ ]:
from openai import OpenAI
import numpy as np

# Initialize OpenAI client
client = OpenAI(api_key="")  #The key in the discord is not working

In [ ]:
# Load and chunk the reference material
with open("messi.txt", "r") as file:
    text = file.read()

chunk_size = 50
chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

# Create embeddings for each chunk
embeddings_dict = {}
responses = client.embeddings.create(
    input=chunks,
    model="text-embedding-3-small"
).data

for i, chunk in enumerate(chunks):
    embeddings_dict[chunk] = np.array(responses[i].embedding)

print(f"Generated embeddings for {len(embeddings_dict)} chunks.")


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:

query = input("Ask a question about Messi: ")

# Create embedding for the query
query_embedding = np.array(
    client.embeddings.create(
        input=query,
        model="text-embedding-3-small"
    ).data[0].embedding
)


Ask a question about Messi: is messi tall?


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************rFMA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
# Function to calculate cosine similarity
def cosine_similarity(A, B):
    return np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))

# Compute similarity scores for each chunk
similarities = [
    (chunk, cosine_similarity(query_embedding, embedding))
    for chunk, embedding in embeddings_dict.items()
]

# Sort chunks by similarity in descending order
similarities = sorted(similarities, key=lambda x: x[1], reverse=True)


top_chunks = [chunk for chunk, _ in similarities[:5]]

print("Top 5 relevant chunks:")
for i, chunk in enumerate(top_chunks, 1):
    print(f"{i}: {chunk}")


In [ ]:
# Combine query and top 5 relevant chunks
augmented_prompt = f"Question: {query}\n\nContext: {' '.join(top_chunks)}"

print("Augmented Prompt:")
print(augmented_prompt)


In [ ]:
# Query OpenAI's completion API
response = client.completions.create(
    model="text-davinci-003",
    prompt=augmented_prompt,
    max_tokens=500
)


print("LLM Response:")
print(response.choices[0].text.strip())
